# Recurrent Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Washington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value.

In [ ]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/mlcollege/introduction-to-ml/master/data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Recurrent Neural Network Regressor

Implement a recurrent neural network regressor. Sort the data by time stamp and deal with it as it was a time series. Be aware of using data from the past as test data.

In [ ]:
data.sort_values(['dteday', 'hr'])
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Add some features from the past

Add the target feature from the previous hour.

In [ ]:
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]

X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed']]
y_all = data['cnt']

data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


### Sequences

Prepare train and test data sequences.

In [ ]:
import numpy as np

window = 50

X_all = X_all.values
y_all = y_all.values

X_seq = []
y_seq = []
for i in range(window, len(X_all) + 1):
    X_seq.append(X_all[i-window: i])
    y_seq.append(y_all[i-1])

X_seq = np.asarray(X_seq)
y_seq = np.asarray(y_seq)

### Split the data into train and test data sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, 
    y_seq,
    random_state=1,
    test_size=0.1)

#split_index = int(X_seq.shape[0]*0.9)
#X_train = X_seq[:split_index,:,:]
#X_test = X_seq[split_index:,:,:]
#y_train = y_seq[:split_index]
#y_test = y_seq[split_index:]

print('Train size: {}'.format(X_train.shape))
print('Test size: {}'.format(X_test.shape))

Train size: (15596, 50, 12)
Test size: (1733, 50, 12)


Standardize the features

In [ ]:
from sklearn.preprocessing import StandardScaler

train_shape = X_train.shape
test_shape = X_test.shape

scaler = StandardScaler()
scaler.fit(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2]))
X_train = scaler.transform(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2])).reshape(train_shape)
X_test = scaler.transform(X_test.reshape(test_shape[0]*test_shape[1], test_shape[2])).reshape(test_shape)

### Training a regressor
Design and train a recurrent regression model with at least one [LSTM](https://keras.io/layers/recurrent/) layer. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/).

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM

model = Sequential()

model.add(LSTM(64, input_shape=(window, 12), return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(64))
model.add(Dropout(0.15))
model.add(Dense(32))
model.add(Dense(1, activation='relu'))

Compile the model

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [ ]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 100, verbose=1,
          validation_data=(X_test, y_test))

Train on 15596 samples, validate on 1733 samples
Epoch 1/100
15596/15596 [==============================] - 30s 2ms/sample - loss: 54326.6725 - mae: 162.6510 - val_loss: 42924.4404 - val_mae: 146.2085
Epoch 2/100
15596/15596 [==============================] - 26s 2ms/sample - loss: 33676.4744 - mae: 135.3574 - val_loss: 30472.4250 - val_mae: 125.0128
Epoch 3/100
15596/15596 [==============================] - 26s 2ms/sample - loss: 18097.0041 - mae: 85.9202 - val_loss: 14277.9186 - val_mae: 75.0910
Epoch 4/100
15596/15596 [==============================] - 26s 2ms/sample - loss: 11244.5203 - mae: 65.4360 - val_loss: 8596.3071 - val_mae: 56.2964
Epoch 5/100
15596/15596 [==============================] - 27s 2ms/sample - loss: 6363.4473 - mae: 48.6715 - val_loss: 5369.1017 - val_mae: 45.6991
Epoch 6/100
15596/15596 [==============================] - 28s 2ms/sample - loss: 4413.7896 - mae: 41.6950 - val_loss: 3871.0100 - val_mae: 39.0619
Epoch 7/100
15596/15596 [===========================

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 199.40911713791112, std: 185.49044320829438
Test Root mean squared error: 28.43
Test Mean absolute error: 18.81


In [ ]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 20.87
Train Mean absolute error: 14.40
